In [9]:
# Dependencies and Setup
# render the figure in a notebook instead of displaying a dump of the figure object.
%matplotlib inline  
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Hide warning messages in notebook
import warnings
warnings.filterwarnings('ignore')

#mouse data : Mouse ID	Drug
mouseData_df= pd.read_csv("data/mouse_drug_data.csv")
#trial data: Mouse ID	Timepoint	Tumor Volume (mm3)	Metastatic Sites
trialData_df= pd.read_csv("data/clinicaltrial_data.csv")

In [38]:
allData_df=pd.merge(mouseData_df,trialData_df, on=["Mouse ID","Mouse ID"])
allData_df.head(3)

,Mouse ID,Drug,Timepoint,Tumor Volume (mm3),Metastatic Sites
0,f234,Stelasyn,0,45.000000,0
1,f234,Stelasyn,5,47.313491,0
2,f234,Stelasyn,10,47.904324,0


In [39]:
#mean Tumor volume data by drug and timepoint
allData_df.set_index(['Drug', 'Timepoint'])
gdata.head()
#gdata= allData_df.groupby(['Drug', 'Timepoint'])
#gdata_df= pd.DataFrame({"Drug":gdata["Drug"], "Timepoint":gdata["Timepoint"], "Tumor Vol":gdata["Tumor Volume (mm3)"].mean()})
#gdata_df= pd.DataFrame( {"Tumor Vol":gdata["Tumor Volume (mm3)"].mean()})
#gdata_df.head()

,Mouse ID,Drug,Timepoint,Tumor Volume (mm3),Metastatic Sites
0,f234,Stelasyn,0,45.000000,0
1,f234,Stelasyn,5,47.313491,0
2,f234,Stelasyn,10,47.904324,0
3,f234,Stelasyn,15,48.735197,1
4,f234,Stelasyn,20,51.112713,2
5,f234,Stelasyn,25,52.178602,2
6,f234,Stelasyn,30,56.873817,2
7,f234,Stelasyn,35,57.748758,2
8,x402,Stelasyn,0,45.000000,0
9,x402,Stelasyn,5,46.327570,1


In [30]:
tgdata_df= gdata_df.transpose()
tgdata_df.head(12)

Timepoint        0                                                         \
Drug      Capomulin Ceftamin Infubinol Ketapril Naftisol Placebo Propriva   
Tumor Vol      45.0     45.0      45.0     45.0     45.0    45.0     45.0   

Timepoint                                ...             45             \
Drug      Ramicane Stelasyn Zoniferol    ...      Capomulin   Ceftamin   
Tumor Vol     45.0     45.0      45.0    ...      36.236114  64.132421   

Timepoint                                                                    \
Drug       Infubinol   Ketapril   Naftisol    Placebo   Propriva   Ramicane   
Tumor Vol  65.755562  70.662958  69.265506  68.084082  66.258529  34.955595   

Timepoint                       
Drug       Stelasyn  Zoniferol  
Tumor Vol  68.43831  65.960888  

[1 rows x 100 columns]